# Submitting form data to NACC Flywheel API



In [ ]:
from getpass import getpass
import logging
import os

from flywheel import Client, FileSpec

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

## Configuring API Access to Flywheel

To use the Flywheel API with the Python SDK you will need an API key.

### STOP! Where are you going to store your API Key?

An API key should be kept secret.

A common way for secrets to be exposed is by hardcoding them into software or configuration files. 
Once the code is checked into a version control repository, the secrets remain in the history even after they are removed from the current version of the code.
So, before you start using your key in your scripts, please be sure you have a good secret management strategy in place.

Here, we just prompt the user for the key.
And, while this avoids storing the key, it is not practical for automated transfers.

### Finding your API key

Each API key is associated with a particular user. 
To get the API key, login as the user to the NACC Flywheel instance.

1. Find the "avatar" in the upper right corner (generally a circle with your initials).
2. Click the avatar dropdown, and select "Profile".
3. Under "Flywheel Access" at the bottom of the resulting page, click "Generate API Key".
4. Choose a key name relevant to upload, set the expiration date, and create the API Key.
5. Copy the API Key since you wont be able to access the value later.
6. Keep the key secret   

### Load secret key in script

In this demonstration script, we just prompt for the key:

In [ ]:
API_KEY = getpass('Enter API_KEY here: ')

## Connecting to Flywheel

Once you have the API Key, it can be used to connect to Flywheel.
With the Python SDK we create an SDK client that we will use throughout the rest of the notebook.

In [ ]:
fw = Client(API_KEY if 'API_KEY' in locals() else os.environ.get('FW_KEY'))

log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

## Figuring out where to upload

### Identify group for center

Each center is associated with a Flywheel group.

Historically, NACC has used an ADC ID to represent centers, but in Flywheel a group has a symbolic ID.
You can find this ID either using a lookup table using the ADCID, or by using the Flywheel user interface to find your group.


Group information is stored as metadata in a Flywheel project `fw://nacc/metadata`.

Get this project with the following code

In [ ]:
metadata = fw.lookup("nacc/metadata")
if not metadata:
    log.error("Failed to find nacc/metadata project")

Each project has an info object, and we've stored the table of centers in 
`metadata.info`.
The SDK doesn't load the info object automatically, so we first call `reload` on the project.


In [ ]:
metadata = metadata.reload()

Then we can access the centers as `metadata.info['centers']`, which is a dictionary for which the keys are ADC IDs.

For instance, to get the entry for the sample center used by NACC, we use the ADC ID 0.

In [ ]:
metadata.info['centers'][0]
# {'acdid': 0, 'name': 'Sample Center', 'group': 'sample-center'}

> Note: we share the IDs for all centers, but you will only be able to access a group if access has been granted to you. Access is granted based on information in the NACC Directory provided by center administrators.

To use this value in the rest of the notebook we set `group_id` to the value for `group`:

In [ ]:
group_id = metadata.info['centers'][0]['group']

### Identify project for upload

Each data type has an "ingest" and "sandbox" Flywheel project.
The "ingest" project is for submitting primary data, and the "sandbox" project is for testing.

For data files that have data for many participants, such as forms, the data is submitted as a CSV where each line is a data record for a participant.
These files are attached to the ingest (or sandbox) project as shown here.

> Data files that have a one to one relationship with participant, such as images, are uploaded differently.


Set `project_prefix` to `sandbox` or `ingest` depending on whether you are uploading test or participant data.
The following code will set the label for the project in the group for your center.

In [ ]:
project_prefix = 'sandbox' # for testing
# project_prefix = 'ingest' # for participant data

project_label = f"{project_prefix}-form"
project_reference = f"{group_id}/{project_label}"

Once the group and ingest project are identified, the SDK client is used to lookup the project.
If the following command fails to find the project, something is wrong with your group and project labels.

In [ ]:
upload_project = fw.lookup(project_reference)
if not upload_project:
    log.error("Failed to find upload project %s", project_reference)

## Upload file

If you have a file on disk, you can upload it directly to the project using code like the following.

In [ ]:
filename = "form-data.csv"
file_path = f"./{filename}"
file_type = 'text/csv'

if upload_project:
    upload_project.upload_file(file_path)
    upload_project.update_file(filename, content_type=file_type)

If, on the otherhand, you generate the file contents in memory, create a `flywheel.FileSpec` object that references the contents and then upload the file.

In [ ]:
contents = "your file contents"
file_spec = FileSpec(filename, contents=contents, content_type=file_type)
if upload_project:
    upload_project.upload_file(file_spec)